In [1]:
from matplotlib import pyplot as plt
plt.rcParams["figure.autolayout"] = True
plt.figure()
manager = plt.get_current_fig_manager()
manager.full_screen_toggle()
plt.show()
plt.rcParams["figure.figsize"] = [17.00, 7.00]
import numpy as np
import pandas as pd

<Figure size 640x480 with 0 Axes>

In [2]:
def NormalizeData(a,b,data):
    mini=a
    maxi=b
    X_std=(data - np.min(data)) / (np.max(data) - np.min(data))
    X_scaled = X_std * (maxi - mini) + mini
    return X_scaled
ROC=[21,22] #Cellular area ID
DC=[1,2,3,4,5,6,7,8,9,10] #% of total map segmet number, to be used as cache size
X=[]
for i in DC:
    X.append(str(i))
NMS=5267 #total map segments traveres from ROC at least once.
NN=[]
for i in range(len(DC)+1):
    if i==0:
        NN.append(' ')
    else:
        NN.append(str(DC[i-1])+'%')

In [3]:
result=np.load('predictedUWP.npy',allow_pickle=True)
P_Test=np.load('map_requests.npy',allow_pickle=True)
abg=[[03,0.001, 0.699],[0.4,0.5,0.1]] #np.load('abg.npy',allow_pickle=True) #list of combinations of MU NU gamma (mu gamma eta as per paper)

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (2264159386.py, line 3)

In [15]:
Z_Test=[]
for el in P_Test:
    Z_Test.append(el[116:]) #80:20 split of map requests
Z_Test=np.array(Z_Test)

146
146


In [21]:
#Overall Maximum Penalty and Hits are calculated
COU=[]
PEN=[]
th=0
for t in range(2):
    L_Test=Z_Test[t]
    c=0
    p=0
    temp=[]
    for l in range(30):
        if DOU[t][l]<th:
            continue
        for j in range(1,7):
            for ul in L_Test[l][0][j]:
                c=c+1
                p=p+ul[1]
                cm=cm+1
        temp.append(cm)
               # print(p)
    COU.append(c)
    PEN.append(p)
    DOU.append(temp)

In [25]:
algo=['UPPN','LRU','DeepCache_with_LFU','LFU','DeepCache'] #Algorithm Names

In [27]:
result=NormalizeData(0,1,result) #Normalization of UWP while using Algorithms

In [28]:
#['UPPN','LRU','DeepCache_with_LFU','LFU','DeepCache']
rand=range(Z_Test.shape[1])
HITDC=[]
PENDC=[]
#HITDC.append(NN)
#PENDC.append(NN)
for t in range(2):
    L_Test=Z_Test[t]
    result1=result[t][116:]
    #pop=popall[t]
    #Don't change avgp
    cache=[]
    for i in DC:
        cache.append(int((i/100)*NMS))
    UPPN=[]
    #UPPN.append('UPPN')
    #LUP=0
    UPPN_Penalty=[]
    #UPPN_Penalty.append('UPPN')
    for c in cache:
        HITVSMISS=[]
        for l in rand:
            if DOU[t][l]<th:
                continue
            hit=0
            miss=0
            sup=0
            for j in range(1,7):
                #random.shuffle(L_Test[l][0][j])
                for ul in L_Test[l][0][j]:
                    i=ul[0]
                    up=ul[1]
                    
                    if i in np.flip(np.argsort(result1[l]))[:c]:
                        hit=hit+1
                        #print("h",i)
                    else:
                        miss=miss+1
                        sup=sup+up
                        #LUP=LUP+up
                        #print(i,up,LUP)
            HITVSMISS.append([hit,miss,sup])
        #print('c:',c)
        sumhit=0
        sumup=0
        for el in HITVSMISS:
           # sumhit=sumhit+(100*el[0]/(sum(el)))
            sumhit=sumhit+(el[0])
            sumup=sumup+el[2]
        #avgp.append(round(sumhit/len(HITVSMISS),3))
        UPPN.append(sumhit)
        UPPN_Penalty.append(sumup)
    HITDC.append(UPPN)
    PENDC.append(UPPN_Penalty)
    cache=[]
    for i in DC:
        cache.append(int((i/100)*NMS))
    LRU=[]
    #LRU.append('LRU')
    LRUM=[]
    LRU_Penalty=[]
    #LRU_Penalty.append('LRU')
    for el in cache:
        capacity = el
        #processList =nl
        Miss = 0
        Hit = 0
        SUP=0
        s=[]
        #print("first",s)
        for l in rand:
            if DOU[t][l]<th:
                continue
           # s = []

            for j in range(1,7):
                #random.shuffle(L_Test[l][0][j])
                for ul in L_Test[l][0][j]:
                    #print(L_Test[l][0][j])
                    i=ul[0]
                    up=ul[1]
                    
                    if i not in s:
                        if(len(s) == capacity):
                            s.remove(s[0])
                            s.append(i)
                        else:
                            s.append(i)
                        Miss +=1
                        SUP=SUP+up
                        #print(i,"m",s)
                    else:
                        Hit+=1
                        s.remove(i)
                        s.append(i)
                        #print(i,"h",s)
        LRU.append(Hit)
        LRUM.append(Miss)
        LRU_Penalty.append(SUP)
    HITDC.append(LRU)
    PENDC.append(LRU_Penalty)
    
    #DeepCache_with_LFU
    cache=[]
    for i in DC:
        cache.append(int((i/100)*NMS))
    DeepCache_with_LFU=[]
    #DeepCache_with_LFU.append('DeepCache_with_LFU')
    DeepCache_with_LFU_Penalty=[]
    #DeepCache_with_LFU_Penalty.append('DeepCache_with_LFU')
    #SUP=0
    for c in cache:
        HITVSMISS=[]
        #s=[]
        #Count=[]
        capacity=c
        for l in rand:
            if DOU[t][l]<th:
                continue
            hit=0
            miss=0
            SUP=0
            s=np.flip(np.argsort(result1[l]))[:c].tolist()
            #s=[]
            Count=[]
            for q in range(c):
                Count.append(0)
            for j in range(1,7):
                temp=[]
                #random.shuffle(L_Test[l][0][j])
                for ul in L_Test[l][0][j]:
                    i=ul[0]
                    up=ul[1]
                    if i not in s:
                        #print("There is miss.", i)
                        if(len(s) == capacity):
                            minpos = Count.index(min(Count))
                            #print("Cache is full so frequent count array:",Count," :minimumfreq index is:",minpos)
                            #print(s[minpos], "will be removed at position:",minpos)
                            del s[minpos]
                            del Count[minpos]
                            s.append(i)
                            Count.append(1)
                        else:
                            s.append(i)
                            #print("Cache is NOT full so pushed",i)
                            Count.append(1)
                        miss=miss+1
                        SUP=SUP+up
                    else:
                        hit=hit+1
                        #s.append(i)
                        ind=s.index(i)
                        tempc=Count[ind] #tempc is not unique tai remove diye korle multiple delete hote pare
                        del Count[ind]
                        del s[ind]
                        Count.append(tempc+1)
                        s.append(i)
                        #print(el[1],"h",s)
                        #print(i," is in cache")
                    #print("Now Cache:",s)
                   # print("Now Count:",Count)
                    #print(".................................")
            HITVSMISS.append([hit,miss,SUP]) 
       # print('c:',c)
        sumhit=0
        sumup=0
        for el in HITVSMISS:
            sumhit=sumhit+el[0]
            sumup=sumup+el[2]
        DeepCache_with_LFU.append(sumhit)
        DeepCache_with_LFU_Penalty.append(sumup)
    HITDC.append(DeepCache_with_LFU)
    PENDC.append(DeepCache_with_LFU_Penalty)
    
    
    #LFU
    cache=[]
    for i in DC:
        cache.append(int((i/100)*NMS))
    LFU=[]
    #LFU.append('LFU')
    LFU_Penalty=[]
    #LFU_Penalty.append('LFU')
    #SUP=0
    for c in cache:
        HITVSMISS=[]
        s=[]
        Count=[]
        capacity=c
        for l in rand:
            if DOU[t][l]<th:
                continue
            hit=0
            miss=0
            SUP=0
            #s=np.flip(np.argsort(result1[l]))[:c].tolist()
            #s=[]
            #Count=[]
            #for q in range(c):
                #Count.append(0)
            for j in range(1,7):
                temp=[]
                #random.shuffle(L_Test[l][0][j])
                for ul in L_Test[l][0][j]:
                    i=ul[0]
                    up=ul[1]
                    if i not in s:
                        #print("There is miss.", i)
                        if(len(s) == capacity):
                            minpos = Count.index(min(Count))
                            #print("Cache is full so frequent count array:",Count," :minimumfreq index is:",minpos)
                            #print(s[minpos], "will be removed at position:",minpos)
                            del s[minpos]
                            del Count[minpos]
                            s.append(i)
                            Count.append(1)
                        else:
                            s.append(i)
                            #print("Cache is NOT full so pushed",i)
                            Count.append(1)
                        miss=miss+1
                        SUP=SUP+up
                       # print(up,SUP)
                        #print(i,ul[1],SUP)
                        #print("m",i,s)
                    else:
                        hit=hit+1
                        #s.append(i)
                        ind=s.index(i)
                        tempc=Count[ind] #tempc is not unique tai remove diye korle multiple delete hote pare
                        del Count[ind]
                        del s[ind]
                        Count.append(tempc+1)
                        s.append(i)
                        #print(el[1],"h",s)
                        #print(i," is in cache")
                    #print("Now Cache:",s)
                   # print("Now Count:",Count)
                    #print(".................................")
            HITVSMISS.append([hit,miss,SUP]) 
       # print('c:',c)
        sumhit=0
        sumup=0
        for el in HITVSMISS:
            sumhit=sumhit+el[0]
            sumup=sumup+el[2]
        LFU.append(sumhit)
        LFU_Penalty.append(sumup)
    HITDC.append(LFU)
    PENDC.append(LFU_Penalty)
    
    #DeepCache
    cache=[]
    for i in DC:
        cache.append(int((i/100)*NMS))
    DeepCache=[]
    #DeepCache.append('DeepCache')
    DeepCache_Penalty=[]
    #DeepCache_Penalty.append('DeepCache')
    for el in cache:
        capacity = el
        Miss = 0
        Hit = 0
        SUP=0
        #print("first",s)
        for l in rand:
            if DOU[t][l]<th:
                continue
            s = np.flip(np.argsort(result1[l]))[:el].tolist()
            for j in range(1,7):
                #random.shuffle(L_Test[l][0][j])
                for ul in L_Test[l][0][j]:
                    i=ul[0]
                    up=ul[1]
                    if i not in s:
                        if(len(s) == capacity):
                            s.remove(s[0])
                            s.append(i)
                        else:
                            s.append(i)
                        Miss +=1
                        SUP=SUP+up
                        #print(i,"m",s)
                    else:
                        Hit+=1
                        s.remove(i)
                        s.append(i)
        #print('c:',el)
        DeepCache.append(Hit)
        DeepCache_Penalty.append(SUP)
    HITDC.append(DeepCache)
    PENDC.append(DeepCache_Penalty)
    #HITDC.append(NN)
    #PENDC.append(NN)
PQR=np.zeros((5, len(DC)))
for u in range(5):
    #for v in range(2):
    for v in range(P_Test.shape[0]):
        PQR[u]=PQR[u]+PENDC[v*5+u]
PQR=PQR*(1/(PEN[0]+PEN[1]))

HQR=np.zeros((5, len(DC)))
for u in range(5):
    #for v in range(2):
    for v in range(P_Test.shape[0]):
        HQR[u]=HQR[u]+HITDC[v*5+u]
HQR=HQR*(1/(COU[0]+COU[1]))

penaltyag=[]
penaltyag.append(NN)
for i in range(5):
    temp=[]
    temp.append(algo[i])
    for j in range(len(DC)):
        temp.append(PQR[i][j])
    penaltyag.append(temp)

hitag=[]
hitag.append(NN)
for i in range(5):
    temp=[]
    temp.append(algo[i])
    for j in range(len(DC)):
        temp.append(HQR[i][j])
    hitag.append(temp)
    
df = pd.DataFrame (hitag)
filepath = 'HIT_Evaluation_Baselines.xlsx'
df.to_excel(filepath, index=False)
df = pd.DataFrame (penaltyag)
filepath ='Penalty_Evaluation_Baselines.xlsx'
df.to_excel(filepath, index=False)

In [ ]:
HITDC=[]
PENDC=[]
for t in range(2):    
    TRY=[]
    TRY_Pen=[]
    L_Test=Z_Test[t]
    #result1=NormalizeData(0,1,result[t])
    result1=result[t][116:]
    for gl in range(len(abg)):
        GAMMA=abg[gl][0]
        NU=abg[gl][1]
        MU=abg[gl][2]
        #if (NU+GAMMA==1 and GAMMA!=1 ) or (NU+MU==1 and MU!=1):
           # print(abg[gl])
           # continue
            
        #print(gl)

        cache=[]
        for i in DC:
            cache.append(int((i/100)*NMS))
        CRITIC=[]
        #CRITIC.append([GAMMA,NU,MU])
        CR_Pen=[]
        #CR_Pen.append([GAMMA,NU,MU])
        for il in range(len(cache)):
            HITVSMISS=[]
            c=cache[il]
            capacity=cache[il]
            #print(c)
            for ll in rand:
                l=ll
                if DOU[t][l]<th:
                    continue
                s=np.flip(np.argsort(result1[l]))[:c].tolist()
                #ss=np.flip(np.argsort(result1[l]))[:c+1].tolist()[-1]
                Cr=[]
                for h in range(c):
                    Cr.append(1)
                Count=[]
                for h in range(c):
                        Count.append(0)
                hit=0
                miss=0
                sup=0
                for j in range(1,7):
                    UWP=[]
                    CC=[]
                    for p in range(len(Cr)):
                        UWP.append((result1[l])[s[p]])
                    U=UWP   
                    if(max(Count)==min(Count)):
                        #CC=Count
                        for cc in range(len(Count)):
                            CC.append(0)
                        
                    else:
                         #CC=Count
                        CC=NormalizeData(min(U),max(U),Count)
                    
                    for p in range(len(Cr)):
                        Cr[p]=CC[p]*GAMMA+NU*Cr[p]+MU*U[p]
                    Count=[]
                    for h in range(c):
                        Count.append(0)
                    for ul in L_Test[ll][0][j]:
                        i=ul[0]
                        up=ul[1]
                        if (i not in s):
                            minpos = Cr.index(min(Cr))
                            del s[minpos]
                            del Cr[minpos]
                            del Count[minpos]
                            s.append(i)
                            Cr.append(result1[l][i])
                            Count.append(1)
                            miss=miss+1
                            sup=sup+up
                        else:
                            hit=hit+1
                            ind=s.index(i)
                            tempc=Cr[ind]
                            tempcount=Count[ind]
                            del Cr[ind]
                            del Count[ind]
                            del s[ind]
                            Cr.append(tempc)
                            Count.append(tempcount+1)
                            s.append(i)
                
                
                HITVSMISS.append([hit,miss,sup])
            sumhit=0
            for el in HITVSMISS:
                sumhit=sumhit+el[0]
            sumup=0
            for el in HITVSMISS:
                sumup=sumup+el[2]
            #print(sumhit)
            CRITIC.append(sumhit)
            CR_Pen.append(sumup)
        #print("gl:",gl)
        TRY.append(CRITIC)
        TRY_Pen.append(CR_Pen)
    for el in TRY:
        HITDC.append(el)
    #HITDC.append(NN)
    for el in TRY_Pen:
        PENDC.append(el)
    #PENDC.append(NN)
PQR=np.zeros((len(abg), len(DC)))
for u in range(len(abg)):
    #for v in range(2):
    for v in range(P_Test.shape[0]):
        PQR[u]=PQR[u]+PENDC[v*len(abg)+u]
PQR=PQR*(1/(PEN[0]+PEN[1]))

HQR=np.zeros((len(abg), len(DC)))
for u in range(len(abg)):
    #for v in range(2):
    for v in range(P_Test.shape[0]):
        HQR[u]=HQR[u]+HITDC[v*len(abg)+u]
HQR=HQR*(1/(COU[0]+COU[1]))

penaltyag=[]
penaltyag.append(NN)
for i in range(len(abg)):
    temp=[]
    temp.append(abg[i])
    for j in range(len(DC)):
        temp.append(PQR[i][j])
    penaltyag.append(temp)

hitag=[]
hitag.append(NN)
for i in range(len(abg)):
    temp=[]
    temp.append(abg[i])
    for j in range(len(DC)):
        temp.append(HQR[i][j])
    hitag.append(temp)   
df = pd.DataFrame (hitag)
filepath = 'HIT_Evaluation_SURC.xlsx'
df.to_excel(filepath, index=False)
df = pd.DataFrame (penaltyag)
filepath = 'Penalty_EValuation_SURC.xlsx'
df.to_excel(filepath, index=False)